<h2>  Applied Data Science Capstone Project </h2> 

This notebook will be mainly used for the capstone project

In [1]:
#print("Hello Capstone Project Course!")

<h2>  Segmenting and Clustering Neighborhoods in Toronto </h2> 

<h3>  Part 1:  </h3> 
<p> To obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe </p>

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Scrapping the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes

In [3]:
#The below url contains html table of postal codes of the city of Toronto.
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"

In [4]:
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

In [5]:
#find all html tables in the web page
tables = soup.find_all('table')

Using Pandas to transform the data in the table on the Wikipedia page into a dataframe.

In [6]:
postal_codes_table = pd.read_html(str(tables[0]), flavor='bs4')[0]
postal_codes_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


<p>Wrangling and cleaning the data</p>

  * Removing cells with a borough that is Not assigned.

In [7]:
postal_codes_table['Borough'].replace('Not assigned', np.nan, inplace = True)
postal_codes_table.dropna(subset=['Borough'], inplace = True)
postal_codes_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


* Combining the rows where one postal code area has more than one neighborhood, into one row with the neighborhoods separated with a comma 

In [8]:
postal_codes_df = postal_codes_table.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
for row in range(len(postal_codes_df)):
    if postal_codes_df['Neighbourhood'].iloc[row] =='Not assigned':
        postal_codes_df['Neighbourhood'].iloc[row]= df['Borough'].iloc[row]

Now we have our dataframe

In [10]:
postal_codes_df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [11]:
postal_codes_df.shape

(103, 3)

In [16]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_70c6ce7f23e04bf78b6f6a714bf0b346 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_70c6ce7f23e04bf78b6f6a714bf0b346 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)



In [17]:
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
postal_codes_coord = postal_codes_df.merge(df_data_1, on = 'Postal Code')

In [15]:
postal_codes_coord

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
